<a href="https://colab.research.google.com/github/SitwalaM/nlp-topic-modelling/blob/develop/MG_topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import numpy as np
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]


In [ ]:
# download stopwords

!python -m nltk.downloader stopwords

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# utility functions from -- https://ourcodingclub.github.io/tutorials/topic-modelling-python/


def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet=re.sub(r'^b\s([RT]+)?','',tweet)  #replace RT-tags
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

# cleaning master function
def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub("b'", '', tweet) # remove the b'
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter
    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    return tweet

<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence \s
<input>:28: DeprecationWarning: invalid escape sequence \s
<input>:13: DeprecationWarning: invalid escape sequence 

In [ ]:
def clean2(text):
    text = str(text).lower()
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^a-z A-Z]', ' ',text)
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'@[A-Za-z0–9]+', '', text) 
    text = re.sub(r' +', ' ', text)
    return text

<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:9: DeprecationWarning: invalid escape sequence \[
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:9: DeprecationWarning: invalid escape sequence \[
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:9: DeprecationWarning: invalid escape sequence \[
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:9: DeprecationWarning: invalid escape sequence \[
<input>:12: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:9: DeprecationWarning: invalid escape sequence \[
<input>:12: DeprecationWarning: invalid escape sequence \w
<input>:4: D

In [ ]:
# download the tweet dataset 

!wget https://dsiwork.s3.amazonaws.com/dataset.csv

--2022-04-05 03:55:05--  https://dsiwork.s3.amazonaws.com/dataset.csv
Resolving dsiwork.s3.amazonaws.com (dsiwork.s3.amazonaws.com)... 52.217.226.57
Connecting to dsiwork.s3.amazonaws.com (dsiwork.s3.amazonaws.com)|52.217.226.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 496370 (485K) [text/csv]
Saving to: ‘dataset.csv’

dataset.csv         100%[===================>] 484.74K  --.-KB/s    in 0.1s    

2022-04-05 03:55:06 (4.34 MB/s) - ‘dataset.csv’ saved [496370/496370]



In [ ]:
data = pd.read_csv("dataset.csv", parse_dates=["date_created"],   encoding="ISO-8859-1")

In [ ]:
data.head()

,id,retweet_count,date_created,tweet
0,1508758968482635778,1092,2022-03-29 10:52:24+00:00,b'A 31-year-old Ugandan traditional healer liv...
1,1508661904192913410,2275,2022-03-29 04:26:42+00:00,"b'Date mein kya rakha hai?\nMarch 29th, a very..."
2,1508815292578816015,563,2022-03-29 14:36:12+00:00,b'Moeletsi Mbeki has accused South Africa\xe2\...
3,1509067192888926208,9,2022-03-30 07:17:10+00:00,"b""RT @ThamiMasemola: Isuzu Motors South Africa..."
4,1509067189827026945,0,2022-03-30 07:17:09+00:00,"b'SAFA President, ANC Dr Danny Jordaan \n\nIs ..."


In [ ]:
#data['clean_tweet'] = data.tweet.apply(clean_tweet)
data['clean_tweet'] = data.tweet.apply(clean2)
data.head()

,id,retweet_count,date_created,tweet,clean_tweet
0,1508758968482635778,1092,2022-03-29 10:52:24+00:00,b'A 31-year-old Ugandan traditional healer liv...,year old ugandan traditional healer living so...
1,1508661904192913410,2275,2022-03-29 04:26:42+00:00,"b'Date mein kya rakha hai?\nMarch 29th, a very...",date mein kya rakha hai nmarch very significa...
2,1508815292578816015,563,2022-03-29 14:36:12+00:00,b'Moeletsi Mbeki has accused South Africa\xe2\...,moeletsi mbeki has accused south africa rulin...
3,1509067192888926208,9,2022-03-30 07:17:10+00:00,"b""RT @ThamiMasemola: Isuzu Motors South Africa...",isuzu motors south africa has begun manufactu...
4,1509067189827026945,0,2022-03-30 07:17:09+00:00,"b'SAFA President, ANC Dr Danny Jordaan \n\nIs ...",safa president anc danny jordaan nis destroyi...


In [ ]:
# Remove stopwords
stop_words = set(stopwords.words("english"))
data["clean_tweet"] = data["clean_tweet"].apply(lambda x : " ".join([w.lower() for w in x.split() if w not in stop_words and len(w) > 3]))

In [ ]:
#Tokenize tweet
tweets = data["clean_tweet"].apply(lambda x : x.split())

In [ ]:
#skipping lemmatization improved the topic results -- needs more investigation

#lemmatize
lemma = WordNetLemmatizer()
nltk.download('wordnet')
lemming_tweets = tweets.apply(lambda x:[lemma.lemmatize(i) for i in x])
lemming_tweets


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


0       [year, ugandan, traditional, healer, living, s...
1       [date, mein, rakha, nmarch, significant, date,...
2       [moeletsi, mbeki, accused, south, africa, ruli...
3       [isuzu, motor, south, africa, begun, manufactu...
4       [safa, president, danny, jordaan, destroying, ...
                              ...                        
2395    [mined, kimberley, south, africa, carat, tiffa...
2396    [breaking, government, mexico, publicly, rejec...
2397    [giveaway, human, lion, cage, experience, life...
2398      [save, exchange, south, africa, showing, flame]
2399    [must, watch, ndrama, agony, ecstasy, access, ...
Name: clean_tweet, Length: 2400, dtype: object

# Modelling 

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.3 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=e86c8838a6db2de2df47b29f2e6b3f41c0f14287f19f1eab6205842046a63ce0
  Stored in directory: /tmp/pip-ephem-wheel-cache-b4p6btkn/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
pip install gensim

In [ ]:
#create id2word dictionary
id2word = Dictionary(lemming_tweets)
print(len(id2word))

3633


In [ ]:
#id2word.filter_extremes(no_below=2, no_above=.99)
#print(len(id2word))

1823


In [ ]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in lemming_tweets]

In [ ]:
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=8, id2word=id2word, workers=12, passes=5)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

In [ ]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
south africa oscar lupita would world afterparty come report worst

------ Topic 1 ------
africa south say manufacturing shift cancer bell away northern equipment

------ Topic 2 ------
south africa country power russia sanction global government western covid

------ Topic 3 ------
nelson africa never mandela south long name happy shall forgotten

------ Topic 4 ------
south africa watch look india must http access epic scene

------ Topic 5 ------
continue friend year white south indian coloured morning medium africa

------ Topic 6 ------
south africa morning brother skill rare fighter african child country

------ Topic 7 ------
south africa woman hockey world know held zimbabwe team junior

------ Topic 8 ------
south africa east asia wheat middle note anything dependent similarly

------ Topic 9 ------
south africa stand leaked call reject text across return globe



In [ ]:
# Instantiating a Base LDA model 

LDA = gensim.models.ldamodel.LdaModel
lda_model = LDA(corpus=corpus, id2word=id2word, num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [ ]:
lda_model.print_topics()

[(0,
  '0.031*"khatronkekhiladi" + 0.021*"away" + 0.020*"time" + 0.020*"manufacturing" + 0.019*"say" + 0.019*"bell" + 0.019*"truck" + 0.019*"shift" + 0.018*"south" + 0.018*"northern"'),
 (1,
  '0.102*"south" + 0.095*"africa" + 0.020*"african" + 0.012*"president" + 0.012*"apartheid" + 0.011*"black" + 0.009*"country" + 0.009*"europe" + 0.009*"america" + 0.008*"nation"'),
 (2,
  '0.038*"woman" + 0.030*"hockey" + 0.029*"country" + 0.024*"held" + 0.022*"world" + 0.022*"stand" + 0.019*"south" + 0.019*"team" + 0.019*"russia" + 0.019*"know"'),
 (3,
  '0.088*"africa" + 0.087*"south" + 0.028*"morning" + 0.028*"brother" + 0.027*"skill" + 0.025*"rare" + 0.025*"fighter" + 0.014*"many" + 0.012*"organized" + 0.012*"solidarity"'),
 (4,
  '0.065*"south" + 0.063*"africa" + 0.044*"india" + 0.042*"must" + 0.040*"look" + 0.039*"watch" + 0.035*"http" + 0.035*"access" + 0.034*"epic" + 0.034*"behind"'),
 (5,
  '0.050*"africa" + 0.049*"south" + 0.041*"continue" + 0.027*"friend" + 0.022*"white" + 0.013*"indian"

**VISUALIZATION**

In [ ]:
pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=2c2c25c56fc708ab077e07bfbace634136a86aff59c88e1c8dd47a371c5fe123
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [ ]:
#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
#from pprint import pprint
#from wordcloud import STOPWORDS
#stopwords = set(STOPWORDS)

visualization of model1

In [ ]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=lemming_tweets, 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -6.489449066841204

Coherence Score:  0.32775855869944387


In [ ]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
lda_viz = gensimvis.prepare(base_model, corpus, id2word)
lda_viz

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.019795 -0.010741       1        1  18.279889
4     -0.250847 -0.037011       2        1  14.197494
8      0.009863 -0.085532       3        1  10.972442
2     -0.063386  0.023428       4        1  10.075355
1     -0.004531  0.016545       5        1   8.857246
9     -0.009419  0.069080       6        1   8.324451
7      0.050134  0.066386       7        1   8.274254
0      0.011478  0.021450       8        1   8.229177
3      0.135057  0.159605       9        1   6.832470
5      0.141447 -0.223212      10        1   5.957222, topic_info=         Term        Freq        Total Category  logprob  loglift
186     oscar  104.000000   104.000000  Default  30.0000  30.0000
141     woman   85.000000    85.000000  Default  29.0000  29.0000
281     watch  109.000000   109.000000  Default  28.0000  28.0000
464    nelson   67.000000    67.000000  Default  27.0000  27.0000
185    lupita   68.000000    68.000000  Default  26.0000  26.0000
..        ...         ...          ...      ...      ...      ...
771  fighting   12.745589    39.496600  Topic10  -4.6416   1.6895
315      time   12.596067    53.835359  Topic10  -4.6534   1.3680
6       south   21.182833  1433.580319  Topic10  -4.1335  -1.3942
0      africa   16.583373  1370.048599  Topic10  -4.3783  -1.5936
51     russia    8.997697    58.196472  Topic10  -4.9898   0.9537

[494 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
174       5  0.856459    absence
730       3  0.053683     abused
730       9  0.107366     abused
730      10  0.805245     abused
2656      2  0.748328   accepted
...     ...       ...        ...
29        7  0.946932   zimbabwe
29       10  0.025593   zimbabwe
732       3  0.062187  zimpapers
732      10  0.932799  zimpapers
1039      5  0.833593     zombie

[947 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 9, 3, 2, 10, 8, 1, 4, 6])

visualization for model2

In [ ]:
pyLDAvis.enable_notebook()
lda_viz = gensimvis.prepare(lda_model, corpus, id2word)
lda_viz

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.058922  0.159445       1        1  14.646697
9      0.079764  0.084889       2        1  12.861522
4      0.113352  0.010723       3        1  11.660896
1      0.073189  0.033852       4        1  10.439003
7     -0.019310  0.008569       5        1  10.080589
2     -0.341137  0.108653       6        1   9.140524
8      0.065142 -0.029238       7        1   8.996384
5      0.004347 -0.128223       8        1   8.641358
3      0.046286 -0.001559       9        1   7.820627
0     -0.080556 -0.247112      10        1   5.712400, topic_info=         Term        Freq        Total Category  logprob  loglift
186     oscar  118.000000   118.000000  Default  30.0000  30.0000
277      look  103.000000   103.000000  Default  29.0000  29.0000
278      must  114.000000   114.000000  Default  28.0000  28.0000
276     india  131.000000   131.000000  Default  27.0000  27.0000
281     watch  101.000000   101.000000  Default  26.0000  26.0000
..        ...         ...          ...      ...      ...      ...
195       say   24.165153    40.451606  Topic10  -3.9599   2.3473
315      time   25.301858    48.204385  Topic10  -3.9139   2.2180
240  nhlanhla   12.088912    22.923821  Topic10  -4.6525   2.2226
6       south   22.949352  1418.921088  Topic10  -4.0115  -1.2618
0      africa   22.498365  1365.891928  Topic10  -4.0313  -1.2436

[453 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
174      10  0.926173    absence
730       7  0.956567     abused
269       3  0.987842     access
635       6  0.851392     across
635       7  0.116099     across
...     ...       ...        ...
805       9  0.916709       zanu
650       7  0.933792    zealand
1669      8  0.902481       zero
29        6  0.990407   zimbabwe
732       7  0.948922  zimpapers

[536 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 10, 5, 2, 8, 3, 9, 6, 4, 1])

In [ ]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=lda_model, texts=lemming_tweets, 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -6.557480079926102

Coherence Score:  0.3445004717085379
